<a href="https://colab.research.google.com/github/yongsa-nut/SF323_CN408_AIEngineer/blob/main/HW11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW11: Recipe Chatbot Evals (5 points)

Adapted from https://github.com/ai-evals-course/recipe-chatbot/tree/main

## Part 1. Write a System Prompt for Thai Recipe Chatbot (0.5 points)

### **Important Note**: Bot ต้องตอบเป็นภาษาไทย

---
  *   **Define the Bot's Role & Objective**: บอกให้ชัดเจนว่า Bot ของคุณคืออะไร (e.g., "You are a friendly and creative culinary assistant specializing in suggesting easy-to-follow recipes.")
  *   **Instructions & Response Rules**: Be specific.
      *   อะไรที่ Bot ต้องทำเสมอ (e.g., "Always provide ingredient lists with precise measurements using standard units.", "Always include clear, step-by-step instructions.")
      *   อะไรที่ Bot ไม่ควรทำ (e.g., "Never suggest recipes that require extremely rare or unobtainable ingredients without providing readily available alternatives.", "Never use offensive or derogatory language.")
  *   **LLM Agency - How Much Freedom?**:
      *   บอกว่าควรให้มันมีความคิดสร้างสรรค์หรือคิดเองประาณไหน (e.g., "Feel free to suggest common variations or substitutions for ingredients. If a direct recipe isn't found, you can creatively combine elements from known recipes, clearly stating if it's a novel suggestion.")
  *   **Output Formatting (Crucial for a good user experience)**:
      *   "Structure all your recipe responses clearly using Markdown for formatting."
      *   "Begin every recipe response with the recipe name as a Level 2 Heading (e.g., `## Amazing Blueberry Muffins`)."
      *   "Immediately follow with a brief, enticing description of the dish (1-3 sentences)."
      *   "Next, include a section titled `### Ingredients`. List all ingredients using a Markdown unordered list (bullet points)."
      *   "Following ingredients, include a section titled `### Instructions`. Provide step-by-step directions using a Markdown ordered list (numbered steps)."
      *   "Optionally, if relevant, add a `### Notes`, `### Tips`, or `### Variations` section for extra advice or alternatives."
  *   **ตัวอย่างของ Markdown structure for a recipe response** (ของคุณต้องเป็นภาษาไทย):

        ```markdown
        ## Golden Pan-Fried Salmon

        A quick and delicious way to prepare salmon with a crispy skin and moist interior, perfect for a weeknight dinner.

        ### Ingredients
        * 2 salmon fillets (approx. 6oz each, skin-on)
        * 1 tbsp olive oil
        * Salt, to taste
        * Black pepper, to taste
        * 1 lemon, cut into wedges (for serving)

        ### Instructions
        1. Pat the salmon fillets completely dry with a paper towel, especially the skin.
        2. Season both sides of the salmon with salt and pepper.
        3. Heat olive oil in a non-stick skillet over medium-high heat until shimmering.
        4. Place salmon fillets skin-side down in the hot pan.
        5. Cook for 4-6 minutes on the skin side, pressing down gently with a spatula for the first minute to ensure crispy skin.
        6. Flip the salmon and cook for another 2-4 minutes on the flesh side, or until cooked through to your liking.
        7. Serve immediately with lemon wedges.

        ### Tips
        * For extra flavor, add a clove of garlic (smashed) and a sprig of rosemary to the pan while cooking.
        * Ensure the pan is hot before adding the salmon for the best sear.
        ```

In [ ]:
recipe_bot_prompt = """เติมคำตอบของคุณที่นี้""" # Fill in your answer here

## Part 2. Logging with LangFuse (0.5 points)

- สำหรับการบ้านนี้เราจะ Log traces ด้วย Langfuse
- **Importance**: ทำเสร็จแล้วให้แนบ sceenshot ของ หน้าหลักของ traces ทั้งหมดมาด้วย (ไม่ต้องโชว์แต่ละtrace)

In [ ]:
!pip install langfuse

In [ ]:
from google.colab import userdata
from langfuse.openai import openai
from langfuse import observe
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."   # เติม secret key ของ LangFuse
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."   # เติม secret key ของ LangFuse
os.environ["LANGFUSE_BASE_URL"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_BASE_URL"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Your openai key
os.environ["OPENAI_API_KEY"] = userdata.get('openrouter')  # เติม openrouter key ของคุณ

In [ ]:
@observe()
def generate(system_prompt, query):
    client = openai.OpenAI(
        base_url="https://openrouter.ai/api/v1",
    )
    completion = client.chat.completions.create(
        name="hw11",
        model="google/gemini-2.5-flash-preview-09-2025",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query}],
        temperature=0,
        metadata={"langfuse_session_id": "session_1"}
    )
    return completion.choices[0].message.content

In [ ]:
# Testing your system prompt
generate(recipe_bot_prompt, 'อยากได้วิธีทำกระเพราะไก่')

## Part 3. Define Dimensions & Generate Initial Queries (2 points)

1. **Identify Key Dimensions**: (i.e., key aspects or variables of user inputs you'll use to generate diverse test queries, such as `cuisine_type`, `dietary_restriction`, or `meal_type` for your recipe bot)

  - สร้าง 3 key dimensions ที่เกี่ยวข้องกับ your Recipe Bot's functionality และ ข้อมูลนำเข้าจาก user.
  - สำหรับแต่ละ dimension ให้สร้าง ตัวอย่างค่าของ dimension นั้นๆ อย่างน้อย สามค่า.

2. **Generate Unique Combinations (Tuples)**:

  - เขียน prompt สำหรับ LLM เพื่อสร้าง 15 unique combinations (tuples) จาก dimensions ด้านบน (1 tuple ต้องมีสาม dimensions)

3. **Generate Natural Language User Queries**:

  - เขียน prompt สำหรับ LLM ที่จะรับ 10 tuples (เลือกเอง) เพื่อสร้าง query จริงออกมาใช้ทดสอบ ให้สร้าง 3 queries ต่อ 1 tuple. รวมทั้งหมด 30 อัน
  - หลังจากนั้นให้เช็คว่า ที่สร้างออกมานั้น อันนั้นสมจริงและคิดว่าเป็นคำถามที่เป็นไปได้บ้าง และเลือกมาใส่ใน dataset ไว้ทดสอบ
  

**คำแนะนำ**: ดูตัวอย่างจาก slide ได้

**Answer**: ตอบ Key Dimensions พร้อมคำอธิบาย ด้านล่าง

1.
2.
3.





In [ ]:
tuple_gen_prompt = """เติม prompt ในนี่"""

# เติม code เพื่อ gen tuple ออกมา 15 อันที่ไม่ซ้ำกัน



uniques_tuple = ...
print(unique_tuple)

In [ ]:
query_prompt = """เติม prompt ที่นี่"""


queries = ...
print(queries)

In [ ]:
dataset = []   # นำ query ที่ผ่านการคัดกรองมาใส่

ลิสต์ queries ที่ไม่ผ่านด้านล่าง และบอกเหตุผลสั้นๆว่าเพราะอะไร

-

## Part 4. Initial Error Analysis (2 points)

1. **Run Bot on Synthetic Queries**:

  - รัน Bot ด้วย System Prompt ที่สร้างไว้สำหรับ Recipe Bot ด้วย dataset จากข้อสาม และเก็บ results ทั้งหมดไว้
  -  ตรวจสอบ trace ใน LangFuse ได้. สามารถ download มา analyze หรือ annotate บน LangFuse ได้


2. **Open Coding**: ในขั้นตอนนี้เราจะมาอ่านและตรวจสอบทุก traces ที่โมเดลเจนมาเพื่อหา patterns หรือ errors โดยที่เรายังไม่จัดหมวดหมู่ของ errors:

  - สำหรับแต่ละ traces เขียนว่ามี themes, patterns, and potential errors or areas for improvement อะไรบ้าง
  - ทำใส่ใน spreadsheet ได้ ให้ส่ง file นี้ด้วย

3. **Axial Coding & Taxonomy Definition**: ในขั้นถัดไปเราจะมาจับกลุ่มของ errors ต่างๆ เป็นกลุ่มย่อยๆ (failure modes) ที่ชัดเจน:

  - ในแต่ละกลุ่ม/modeนั้น สร้าง taxonomy ที่ชัดเจนที่ประกอบไปด้วย:
     - Title
     - คำอธิบายของ failure mode นี้
     - ตัวอย่างของ failure mode นี้ 1 - 2 ตัวอย่าง

**คำตอบ ของ Open Coding**:

--- เติมคำตอบด้านล่าง หรือ ส่งไฟล์ cvs/speedsheet แยกมาต่างหาก ---



**คำตอบ Axial Coding & Taxonomy Definition**:

--- เติมคำตอบด้านล่าง ---